In [1]:
import pandas as pd
import glob
import math
import numpy as np

# Import Data Order Blibli from Email

In [2]:
orderblibli=pd.concat([pd.read_csv(f, dtype=str) for f in glob.glob("E:\REPORTING PARTNER IDX\PROJECT BLIBLI\DATABASE BLIBLI\ORDER FROM EMAIL\Order email\*.csv")], ignore_index = True)
#orderblibli.loc[:,["Order Status"]]
orderblibli=orderblibli.sort_values(by = ["Airway Bill No."], ascending=[False])
orderblibli.drop_duplicates(subset=['Airway Bill No.'], keep='first', inplace = True)
orderblibli.dropna(axis=0,how='all', inplace=True)
orderblibli.rename(columns={"Airway Bill No.":"Waybill No."},inplace=True)
orderblibli.loc[pd.notna(orderblibli["Warehouse Name"]),"Order Source Blibli"]="Warehouse"
orderblibli.loc[pd.isna(orderblibli["Warehouse Name"]),"Order Source Blibli"]="Seller"
# orderblibli.info()

In [3]:
orderblibli.to_excel("E:\REPORTING PARTNER IDX\PROJECT BLIBLI\Blibli Python Template Results/02052023.data_email_Blibli_Result.xlsx", index=False)

# Import Raw data order Blibli from Big Query (format csv)

In [4]:
search_order=pd.concat([pd.read_csv(f, dtype=str) for f in glob.glob("E:\REPORTING PARTNER IDX\PROJECT BLIBLI\DATABASE BLIBLI\DATA ORDER BLIBLI\*.csv")], ignore_index = True)

search_order.rename(columns={"Waybill_No":"Waybill No."},inplace=True)
search_order.rename(columns={"Order_No":"Order No."},inplace=True)
search_order.rename(columns={"Sender_Phone":"Sender_Phone"},inplace=True)
search_order.rename(columns={"Origin_Province":"Origin Province"},inplace=True)
search_order.rename(columns={"Origin_City":"Origin City"},inplace=True)
search_order.rename(columns={"Origin_District":"Origin District"},inplace=True)
search_order.rename(columns={"Pickup_Area":"Pick Up Area"},inplace=True)
search_order.rename(columns={"Sender_Address":"Sender Address"},inplace=True)
search_order.rename(columns={"Scheduled_Branch":"Scheduled Branch"},inplace=True)
search_order.rename(columns={"Pickup_PDB":"Pickup PDB"},inplace=True)

search_order=search_order.sort_values(by = ["Waybill No."], ascending=[False])
search_order.drop_duplicates(subset=['Waybill No.'], keep='first', inplace = True)
search_order.dropna(axis=0, how='all', inplace=True)
#search_order.loc[search_order["Order_Status"]=="Schedule To Courier"

                 
search_order.loc[pd.isna(search_order["Pickup_Time"])&pd.isna(search_order["Pickup_Failure_Reason"]),"Late_Pickup_Reason"]="Dalam Penjadwalan Kurir"
search_order.loc[(search_order["Pickup_Perform"]=="Late")&pd.notna(search_order["Pickup_Time"])&pd.isna(search_order["Pickup_Failure_Reason"]),"Late_Pickup_Reason"]="Dalam Penjadwalan Kurir"
                 
# search_order.info()

In [5]:
# Mapping Late Factor & Late Category

# Merging Data Order from Email & Raw Data Order from BigQuery

In [6]:
main=orderblibli.merge(search_order,how="left", on="Waybill No.")
main.drop_duplicates(subset=['Waybill No.'], keep='first', inplace = True)
main.dropna(axis=0, how='all', inplace=True)

#main.loc[(main["Pickup_Time"] <= main["Tanggal Email"]),"Pickup Performance"]="On Time"
#main.loc[(main["Pickup_Time"] > main["Tanggal Email"]),"Pickup Performance"]="Late"
#main.loc[pd.isna(main["Pickup_Time"]),"Pickup Performance"]="Late"

main.loc[pd.notna(main["Pickup_Time"]),"Pick Up Status"]="Picked Up"
main.loc[main["Order_Status"]=="Picked Up", "Pick Up Status"]="Picked Up"
main.loc[main["Order_Status"]=="Cancel Order", "Pick Up Status"]="Not Yet Pick Up"
main.loc[main["Order_Status"]=="Schedule To Branch", "Pick Up Status"]="Not Yet Pick Up"
main.loc[main["Order_Status"]=="Schedule To Courier", "Pick Up Status"]="Not Yet Pick Up"

main.loc[pd.notna(main["Pickup_Time"]),"Final Status Inbound"]="Picked Up"
main.loc[main["Order_Status"]=="Picked Up", "Final Status Inbound"]="Picked Up"
main.loc[main["Order_Status"]=="Cancel Order", "Final Status Inbound"]="Cancel Order"
main.loc[main["Order_Status"]=="Schedule To Branch", "Final Status Inbound"]="Not Pick up Yet"
main.loc[main["Order_Status"]=="Schedule To Courier", "Final Status Inbound"]="Not Pick up Yet"

#main.loc[pd.isna(main["Pickup_Failure_Reason"]),"Late_Pickup_Reason"]="Dalam penjadwalan kurir"
#main.loc[main["Late_Pickup_Reason"]=="Dalam penjadwalan kurir", "Final Status Inbound"]="Not Pick up Yet"

# main.info()

In [7]:
main.rename(columns={"Tanggal Pengambilan":"Request Pick Up Date"},inplace=True)
main.rename(columns={"Nama Merchant":"Seller Name"},inplace=True)
main.rename(columns={"Alamat Origin (line 1 & 2)":"Seller Address"},inplace=True)
main.rename(columns={"Nomor Telepon Merchant PIC":"Seller Phone"},inplace=True)
main.rename(columns={"Propinsi Pick Up Point":"Origin"},inplace=True)
main.rename(columns={"Pickup_Time":"Actual Pick Up Date"},inplace=True)
main.rename(columns={"Pickup_Time_A":"Actual Pick Up Hour"},inplace=True)
main.rename(columns={"Pickup_Duration_Day":"Pickup Duration-Hour"},inplace=True)
main.rename(columns={"Late_Pickup_Reason":"Pickup Failure Reason"},inplace=True)
main.rename(columns={"Order_Time":"Order Generated Time"},inplace=True)
main.rename(columns={"Late_Factor":"Late Factor"},inplace=True)
main.rename(columns={"Late_Category":"Late Category"},inplace=True)
# main.rename(columns={"Pickup_Area":"Pick Up Area"},inplace=True)

main.loc[main["Final Status Inbound"]=="Cancel Order", "Delivery Status"]="Cancel Order"
main.loc[main["Final Status Inbound"]=="Not Pick up Yet", "Delivery Status"]="Pending Pick Up"
main.loc[pd.notna(main["Shipping_Time"])&pd.isna(main["POD_Time"])&pd.isna(main["Return_Start_Time"])&pd.isna(main["Return_POD_Time"]),"Delivery Status"]="Delivery Process"
main.loc[pd.notna(main["Shipping_Time"])&pd.notna(main["POD_Time"])&pd.isna(main["Return_Start_Time"])&pd.isna(main["Return_POD_Time"]),"Delivery Status"]="Delivered"
main.loc[pd.notna(main["Shipping_Time"])&pd.isna(main["POD_Time"])&pd.notna(main["Return_Start_Time"])&pd.isna(main["Return_POD_Time"]),"Delivery Status"]="Return Process"
main.loc[pd.notna(main["Shipping_Time"])&pd.isna(main["POD_Time"])&pd.notna(main["Return_Start_Time"])&pd.notna(main["Return_POD_Time"]),"Delivery Status"]="Returned to WH/Seller"
main.loc[pd.notna(main["Shipping_Time"])&pd.isna(main["POD_Time"])&pd.isna(main["Return_Start_Time"])&pd.notna(main["Return_POD_Time"]),"Delivery Status"]="Returned to WH/Seller"
main.loc[pd.notna(main["Shipping_Time"])&pd.notna(main["POD_Time"])&pd.notna(main["Return_Start_Time"])&pd.isna(main["Return_POD_Time"]),"Delivery Status"]="Delivered"

# main.info()

In [8]:
#Mapping Kolom Untuk Pickup Report

pickup_report=main.loc[:,["Waybill No.","Package ID","Origin","Seller Name","Seller Address","Seller Phone","Order Source Blibli",
"Request Pick Up Date","Tanggal Email","Pick Up Status",
"Pickup Failure Reason","Final Status Inbound",
"Actual Pick Up Date",
"Actual Pick Up Hour",
"Pickup Duration-Hour",
"Pickup_Perform","Late Factor","Late Category","Pick Up Area","Order Generated Time"]]

pickup_report

,Waybill No.,Package ID,Origin,Seller Name,Seller Address,Seller Phone,Order Source Blibli,Request Pick Up Date,Tanggal Email,Pick Up Status,Pickup Failure Reason,Final Status Inbound,Actual Pick Up Date,Actual Pick Up Hour,Pickup Duration-Hour,Pickup_Perform,Late Factor,Late Category,Pick Up Area,Order Generated Time
0,IDB904294948777,50504693,Jawa Barat,Atelier Angelina Official Store,Warehouse Blibli Cikarang | PT Global Digital ...,081585251540,Warehouse,16/02/2023,17/02/2023,Picked Up,NaN,Picked Up,16-02-23 19:25,2023-02-16 12:25:47 UTC,-1,On Time,NaN,NaN,BEKASI,16-02-23 14:45
1,IDB904294872229,50503940,Jawa Barat,Tira Smart Family Beauty Official Store,Gudang Blibli.com\nKomplek Pergudangan Widya S...,081585251648,Warehouse,16/02/2023,16/02/2023,Picked Up,Pengirim sedang mempersiapkan paket,Picked Up,16-02-23 17:05,2023-02-16 10:05:59 UTC,0,On Time,NaN,NaN,BEKASI,16-02-23 12:36
2,IDB904294585959,50867261,Jawa Barat,IndoKidzShop Official Store,Gudang Blibli.com\nKomplek Pergudangan Widya S...,081585251648,Warehouse,05/03/2023,05/03/2023,Picked Up,NaN,Picked Up,05-03-23 12:04,2023-03-05 05:04:38 UTC,0,On Time,NaN,NaN,BEKASI,05-03-23 08:45
3,IDB904293019204,3824355150,DKI Jakarta,JBL Official Store,Warehouse Titipaja Pluit. \nJl. Pluit Karang K...,6285262000057,Seller,24/03/2023,24/03/2023,Not Yet Pick Up,Dalam Penjadwalan Kurir,Not Pick up Yet,NaN,NaN,NaN,Late,IDE,Controllable,JAKARTA MA1,24-03-23 13:13
4,IDB904292407277,50464603,Jawa Barat,Asih Sadulur Mart,"Jl. Ibrahim Adjie No.338, Kelurahan Binong, Ke...",081585251478,Warehouse,14/02/2023,14/02/2023,Picked Up,Pengirim sedang mempersiapkan paket,Picked Up,14-02-23 18:58,2023-02-14 11:58:59 UTC,0,On Time,NaN,NaN,BANDUNG,14-02-23 10:16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19827,IDB000002690490,49783350,Jawa Barat,Atelier Angelina Official Store,Warehouse Blibli Cikarang | PT Global Digital ...,081585251540,Warehouse,09/01/2023,09/01/2023,Picked Up,Pengirim sedang mempersiapkan paket,Picked Up,09-01-23 17:29,2023-01-09 10:29:00 UTC,0,On Time,NaN,NaN,BEKASI,09-01-23 13:12
19828,IDB000002081259,49958690,DKI Jakarta,Bintang Ponselindo FBB,Jalan Dewi Sartika no 184\nKramat Jati\nCawang,081585251475,Warehouse,19/01/2023,19/01/2023,Picked Up,Pengirim sedang mempersiapkan paket,Picked Up,19-01-23 11:13,2023-01-19 04:13:10 UTC,0,On Time,NaN,NaN,JAKARTA MA4,19-01-23 07:56
19829,IDB000001575420,49768720,Jawa Barat,Buttonscarves Muslim Official Store,Warehouse Blibli Cikarang | PT Global Digital ...,081585251540,Warehouse,08/01/2023,08/01/2023,Picked Up,Pengirim sedang mempersiapkan paket,Picked Up,08-01-23 21:04,2023-01-08 14:04:01 UTC,0,On Time,NaN,NaN,BEKASI,08-01-23 14:09
19830,IDB000001388240,49913389,Jawa Timur,Greensmart FBB Official Store,"Jl. Berbek Industri I Kav 5-19, Desa Berbek, K...",081585251480,Warehouse,16/01/2023,17/01/2023,Picked Up,NaN,Picked Up,16-01-23 19:51,2023-01-16 12:51:23 UTC,-1,On Time,NaN,NaN,SURABAYA,16-01-23 17:05


In [9]:
pickup_report.to_excel("E:\REPORTING PARTNER IDX\PROJECT BLIBLI\Blibli Python Template Results\Pickup Report/27032023.Pickup_Report_Blibli_Result.xlsx", index=False)

In [10]:
#Mapping Kolom Untuk Delivery Report

delivery_report=main.loc[:,["Waybill No.","Package ID","Request Pick Up Date","Shipping_Time","Delivery Status",
"Return_Start_Time","Return_POD_Time","POD_Time","POD_Signer","Relation","Tanggal Email"]]

delivery_report                            

,Waybill No.,Package ID,Request Pick Up Date,Shipping_Time,Delivery Status,Return_Start_Time,Return_POD_Time,POD_Time,POD_Signer,Relation,Tanggal Email
0,IDB904294948777,50504693,16/02/2023,16-02-23 19:25,Delivered,NaN,NaN,18-02-23 11:46,Koni Hasan,NaN,17/02/2023
1,IDB904294872229,50503940,16/02/2023,16-02-23 17:05,Delivered,NaN,NaN,18-02-23 18:24,Kiss Cell mbk Bety Dwi Pratiwi,NaN,16/02/2023
2,IDB904294585959,50867261,05/03/2023,05-03-23 12:04,Delivered,NaN,NaN,06-03-23 12:26,Resmi .,NaN,05/03/2023
3,IDB904293019204,3824355150,24/03/2023,NaN,Pending Pick Up,NaN,NaN,NaN,NaN,NaN,24/03/2023
4,IDB904292407277,50464603,14/02/2023,14-02-23 18:58,Delivered,NaN,NaN,16-02-23 11:21,Rima Astriana,NaN,14/02/2023
...,...,...,...,...,...,...,...,...,...,...,...
19827,IDB000002690490,49783350,09/01/2023,09-01-23 17:29,Delivered,NaN,NaN,11-01-23 10:47,SULIS TIYORINI,NaN,09/01/2023
19828,IDB000002081259,49958690,19/01/2023,19-01-23 11:13,Delivered,NaN,NaN,21-01-23 16:35,sumaji .,NaN,19/01/2023
19829,IDB000001575420,49768720,08/01/2023,08-01-23 21:04,Delivered,NaN,NaN,10-01-23 09:19,Jennifer .,NaN,08/01/2023
19830,IDB000001388240,49913389,16/01/2023,16-01-23 19:51,Delivered,NaN,NaN,18-01-23 11:23,Cafrina ulfa,NaN,17/01/2023


In [11]:
delivery_report.to_excel("E:\REPORTING PARTNER IDX\PROJECT BLIBLI\Blibli Python Template Results\Delivery Report/27032023.Delivery_Report_Blibli_Result.xlsx", index=False)

In [12]:
#Mapping All Report Column (Pickup-Delivery)

main_report=main.loc[:,["Waybill No.","Package ID","Origin","Seller Name","Seller Address","Order Source Blibli",
"Request Pick Up Date","Tanggal Email","Pick Up Status",
"Pickup Failure Reason","Final Status Inbound",
"Actual Pick Up Date",
"Actual Pick Up Hour",
"Pickup Duration-Hour",
"Pickup_Perform","Late Factor","Late Category","Pick Up Area","Order Generated Time","Delivery Status",
"Return_Start_Time","Return_POD_Time","POD_Time","POD_Signer","Relation","Warehouse Name"]]

main_report

,Waybill No.,Package ID,Origin,Seller Name,Seller Address,Order Source Blibli,Request Pick Up Date,Tanggal Email,Pick Up Status,Pickup Failure Reason,...,Late Category,Pick Up Area,Order Generated Time,Delivery Status,Return_Start_Time,Return_POD_Time,POD_Time,POD_Signer,Relation,Warehouse Name
0,IDB904294948777,50504693,Jawa Barat,Atelier Angelina Official Store,Warehouse Blibli Cikarang | PT Global Digital ...,Warehouse,16/02/2023,17/02/2023,Picked Up,NaN,...,NaN,BEKASI,16-02-23 14:45,Delivered,NaN,NaN,18-02-23 11:46,Koni Hasan,NaN,BKI E - Cikarang
1,IDB904294872229,50503940,Jawa Barat,Tira Smart Family Beauty Official Store,Gudang Blibli.com\nKomplek Pergudangan Widya S...,Warehouse,16/02/2023,16/02/2023,Picked Up,Pengirim sedang mempersiapkan paket,...,NaN,BEKASI,16-02-23 12:36,Delivered,NaN,NaN,18-02-23 18:24,Kiss Cell mbk Bety Dwi Pratiwi,NaN,BKI NW - Medan Satria
2,IDB904294585959,50867261,Jawa Barat,IndoKidzShop Official Store,Gudang Blibli.com\nKomplek Pergudangan Widya S...,Warehouse,05/03/2023,05/03/2023,Picked Up,NaN,...,NaN,BEKASI,05-03-23 08:45,Delivered,NaN,NaN,06-03-23 12:26,Resmi .,NaN,BKI NW - Medan Satria
3,IDB904293019204,3824355150,DKI Jakarta,JBL Official Store,Warehouse Titipaja Pluit. \nJl. Pluit Karang K...,Seller,24/03/2023,24/03/2023,Not Yet Pick Up,Dalam Penjadwalan Kurir,...,Controllable,JAKARTA MA1,24-03-23 13:13,Pending Pick Up,NaN,NaN,NaN,NaN,NaN,NaN
4,IDB904292407277,50464603,Jawa Barat,Asih Sadulur Mart,"Jl. Ibrahim Adjie No.338, Kelurahan Binong, Ke...",Warehouse,14/02/2023,14/02/2023,Picked Up,Pengirim sedang mempersiapkan paket,...,NaN,BANDUNG,14-02-23 10:16,Delivered,NaN,NaN,16-02-23 11:21,Rima Astriana,NaN,BDG E - Bandung Kiaracondong
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19827,IDB000002690490,49783350,Jawa Barat,Atelier Angelina Official Store,Warehouse Blibli Cikarang | PT Global Digital ...,Warehouse,09/01/2023,09/01/2023,Picked Up,Pengirim sedang mempersiapkan paket,...,NaN,BEKASI,09-01-23 13:12,Delivered,NaN,NaN,11-01-23 10:47,SULIS TIYORINI,NaN,BKI E - Cikarang
19828,IDB000002081259,49958690,DKI Jakarta,Bintang Ponselindo FBB,Jalan Dewi Sartika no 184\nKramat Jati\nCawang,Warehouse,19/01/2023,19/01/2023,Picked Up,Pengirim sedang mempersiapkan paket,...,NaN,JAKARTA MA4,19-01-23 07:56,Delivered,NaN,NaN,21-01-23 16:35,sumaji .,NaN,CGK SE - Kramat Jati
19829,IDB000001575420,49768720,Jawa Barat,Buttonscarves Muslim Official Store,Warehouse Blibli Cikarang | PT Global Digital ...,Warehouse,08/01/2023,08/01/2023,Picked Up,Pengirim sedang mempersiapkan paket,...,NaN,BEKASI,08-01-23 14:09,Delivered,NaN,NaN,10-01-23 09:19,Jennifer .,NaN,BKI E - Cikarang
19830,IDB000001388240,49913389,Jawa Timur,Greensmart FBB Official Store,"Jl. Berbek Industri I Kav 5-19, Desa Berbek, K...",Warehouse,16/01/2023,17/01/2023,Picked Up,NaN,...,NaN,SURABAYA,16-01-23 17:05,Delivered,NaN,NaN,18-01-23 11:23,Cafrina ulfa,NaN,SUB S - Sidoardjo


In [13]:
#Export All Report Column (Pickup-Delivery) to Excel

main_report.to_excel("E:\REPORTING PARTNER IDX\PROJECT BLIBLI\Blibli Python Template Results\Main Report Pickup-Delivery/27032023.Main Report Pickup-Delivery_Blibli_Result.xlsx", index=False)